In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# objects to build workflow using graphs
from langgraph.graph import StateGraph, MessagesState, START

# in memory checkpointer that enables the app
# to remember past conversations
from langgraph.checkpoint.memory import MemorySaver

ret = load_dotenv()

# returns True if .env loaded successfully
print(ret)

True


In [2]:
model = ChatOpenAI(model_name="gpt-4o-mini")
model_response = model.invoke("Tell me about Dexcom Stelo")
print(model_response)

content='As of my last knowledge update in October 2021, Dexcom had not yet released a product specifically named "Dexcom Stelo." However, it’s possible that developments may have occurred after that date, including new products, expansions, or updates from Dexcom, a company known for its continuous glucose monitoring (CGM) systems for people with diabetes.\n\nDexcom is widely recognized for its CGM systems, which provide real-time glucose readings that help users manage their diabetes more effectively. Their products typically include features such as smartphone compatibility, alarms for high or low blood glucose levels, and data sharing capabilities with caregivers or family members.\n\nFor the most accurate and current information regarding the Dexcom Stelo or any other new product releases, I recommend checking the official Dexcom website or consulting recent news articles and press releases from the company.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'

In [19]:
# define the function that calls the model
def call_model(state: MessagesState):
    # state = current state of the graph, which is a list of messages
    # state["messages"] is a list of all historical messages
    updated_messages = model.invoke(state["messages"])

    # models response appended to the existing list of messages
    return {"messages": updated_messages}

In [20]:
# workflow is a stategraph, keeps track of the state of the application
# we need to use a schema, MessagesState helps to keep track a list
# of messages. in other words, we are saying the state the worflow
# gonna keep track of is a list of messages
workflow = StateGraph(MessagesState)

# nodes take as input the current state of the graph
# they do something and then they update the state
workflow.add_node("model_node", call_model)

# after START, run the model node which calls the model
# add an edge from the START node to the model node
# edges determine which node to execute next, based on current state
workflow.add_edge(START, "model_node")

In [21]:
# real apps will use Postgres or SQlite checkpointers
# for experimentation use in memory checkpointer
# checkpointers store snapshots of past conversation
memory = MemorySaver()
app = workflow.compile(memory)

In [22]:
# app is now a runnable object so has invoke method
# because schema used is MessagesState, input expected
# is dict of {'messages': ""}

chat1 = {'configurable': {'thread_id': 1}}
app.invoke({"messages": "Hi My name is Mahmud!"}, config=chat1)

{'messages': [HumanMessage(content='Hi My name is Mahmud!', additional_kwargs={}, response_metadata={}, id='1d7649b1-a1c8-42c3-83e3-78893b37f699'),
  AIMessage(content='Hi Mahmud! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 14, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'finish_reason': 'stop', 'logprobs': None}, id='run-32306904-4535-4905-a7ff-2c2ab97ba71d-0', usage_metadata={'input_tokens': 14, 'output_tokens': 12, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

In [23]:
# look at the reply. the whole conversation is here
app.invoke({"messages": "What is my name!"}, config=chat1)

{'messages': [HumanMessage(content='Hi My name is Mahmud!', additional_kwargs={}, response_metadata={}, id='1d7649b1-a1c8-42c3-83e3-78893b37f699'),
  AIMessage(content='Hi Mahmud! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 14, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'finish_reason': 'stop', 'logprobs': None}, id='run-32306904-4535-4905-a7ff-2c2ab97ba71d-0', usage_metadata={'input_tokens': 14, 'output_tokens': 12, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content='What is my name!', additional_kwargs={}, response_metadata={}, 

In [24]:
output = app.invoke(None, config=chat1)
for message in output["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi My name is Mahmud!
================================== Ai Message ==================================

Hi Mahmud! How can I assist you today?
================================ Human Message =================================

What is my name!
================================== Ai Message ==================================

Your name is Mahmud! How can I help you further?


In [25]:
# can the app support multiple independant conversations?
chat2 = {'configurable': {'thread_id': 2}}
app.invoke({"messages": "What is my name!"}, config=chat2)["messages"][-1].content

"I don't know your name. If you tell me, I can use it in our conversation!"

This time the app doesnt know my name confirming this is an entirely different chat

In [26]:
# make the app more interactive
def chatbot(chat_id: int):
    # this config dict is required because we are using a checkpointer
    config = {"configurable": {"thread_id": chat_id}}

    while True:
        user_input = input("User: ")
        if user_input in ('exit', 'quit'):
            print("Goodbye")
            break
        else:
            print("Chatbot reply:\n")
            for chunk, metadata in app.stream({'messages': user_input},
                                              config=config,
                                              stream_mode="messages"):
                print(chunk.content, end="", flush=True)
            print("\n")


In [27]:
chatbot(2)

Chatbot reply:

Nice to meet you, Mahmud Hasan! How can I assist you today?

Chatbot reply:

Las Vegas is famous for a variety of reasons, including:

1. **Casinos and Gambling**: Las Vegas is known as the gambling capital of the world, with numerous casinos offering a wide range of gaming options.

2. **Entertainment**: The city is renowned for its entertainment options, including world-class shows, concerts, magic acts, and performances by famous artists.

3. **Nightlife**: Las Vegas boasts a vibrant nightlife scene, with many nightclubs, bars, and lounges that attract visitors from around the globe.

4. **Resorts and Hotels**: The Strip is lined with extravagant hotels and resorts, each offering unique themes and attractions, such as the Bellagio, Caesars Palace, and The Venetian.

5. **Dining**: Las Vegas offers a diverse culinary scene, with numerous fine dining restaurants run by celebrity chefs, buffets, and international cuisine.

6. **Shopping**: The city features luxury shopp

# Adding a system prompt

In [28]:
# System messages are used to direct the behavior of the LLM.We will use chat prompt templates to send system messages

# ChatPromptTemplate =  a reusable structure for creating prompts for chat apps
from langchain_core.prompts import ChatPromptTemplate

In [29]:
# tuples of message types and messages
prompt = ChatPromptTemplate(
    [
        ("system", "Limit all of your responses to two sentences"),
        ("placeholder", "{messages}")  # telling the prompt template
        # is expected to get a variable messages when invoked. this variable will
        # come from the state that is passed to the call function
    ]
)

In [30]:
# state is a dict with key messages and value a list of HumanMessages
state = {"messages": ["What is the history of Delta Airlines"]}

In [31]:
# prompt also has a invoke model
prompt.invoke(state)

ChatPromptValue(messages=[SystemMessage(content='Limit all of your responses to two sentences', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the history of Delta Airlines', additional_kwargs={}, response_metadata={})])

In [32]:
model.invoke(prompt.invoke(state))

AIMessage(content='Delta Airlines was founded in 1925 as a crop-dusting service called Huff Daland Dusters. It transitioned to passenger airline operations in 1929 and expanded significantly through mergers and acquisitions, becoming one of the largest airlines in the world.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 26, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'finish_reason': 'stop', 'logprobs': None}, id='run-cc016ee3-1218-4449-95ea-ca0ed750b1fa-0', usage_metadata={'input_tokens': 26, 'output_tokens': 50, 'total_tokens': 76, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# Chaining multiple actions

In [34]:
# this is a chain of actions = model.invoke(prompt.invoke(state))
# run the steps in order from left to right
# chains are like scikit learn pipelines
chain = prompt | model
chain.invoke(state)

AIMessage(content='Delta Airlines was founded in 1924 as a crop-dusting operation called Huff Daland Dusters and later transitioned to passenger flights in 1929. Over the decades, it expanded through numerous mergers and acquisitions, becoming one of the largest and most significant airlines in the world.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 26, 'total_tokens': 83, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'finish_reason': 'stop', 'logprobs': None}, id='run-ea74da98-d0d2-47c4-978b-9efbfa21f748-0', usage_metadata={'input_tokens': 26, 'output_tokens': 57, 'total_tokens': 83, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reason

# Integrating prompt in workflow code

In [35]:
# define the function that calls the model
def call_model(state: MessagesState):
    # state = current state of the graph, which is a list of messages
    # state["messages"] is a list of all historical messages
    chain = prompt | model
    updated_messages = chain.invoke(state)

    # models response appended to the existing list of messages
    return {"messages": updated_messages}

In [36]:
workflow = StateGraph(MessagesState)
workflow.add_node("model_node", call_model)
workflow.add_edge(START, "model_node")
memory = MemorySaver()
app = workflow.compile(memory)

In [37]:
# now we have a chatbot that onyl gives short answerss
chatbot(2)

Chatbot reply:

Mahmud Hasan may refer to various individuals; could you specify which one or provide more context? Notable figures with that name could include politicians, academics, or athletes.

Chatbot reply:

Mahmud Hasan is a prominent Bangladeshi politician and a member of the Awami League party. He has been involved in various political activities and has served in different capacities within the government.

Chatbot reply:

The Awami League is one of the major political parties in Bangladesh, founded in 1949. It played a significant role in the country's struggle for independence and is known for advocating secularism, socialism, and democracy.

Chatbot reply:

The Bangladesh Nationalist Party (BNP) is a major political party in Bangladesh, founded in 1978 by former President Ziaur Rahman. The party is known for its nationalist ideology and has been a significant political rival to the Awami League, often advocating for conservative and pro-Islamist policies.

Chatbot reply:


KeyboardInterrupt: Interrupted by user

# Translation to Spanish

In [38]:
model = ChatOpenAI(model_name="gpt-4o-mini")

# change in the prompt
prompt = ChatPromptTemplate(
    [
        ("system", "Translate the input from English to Spanish"),
        ("placeholder", "{messages}")  # telling the prompt template
        # is expected to get a variable messages when invoked. this variable will
        # come from the state that is passed to the call function
    ]
)


def call_model(state: MessagesState):
    # state = current state of the graph, which is a list of messages
    # state["messages"] is a list of all historical messages
    chain = prompt | model
    updated_messages = chain.invoke(state)

    # models response appended to the existing list of messages
    return {"messages": updated_messages}


workflow = StateGraph(MessagesState)
workflow.add_node("model_node", call_model)
workflow.add_edge(START, "model_node")
memory = MemorySaver()
app = workflow.compile(memory)

In [39]:
chatbot(3)

Chatbot reply:

hola

Chatbot reply:

¿Cuál es tu nombre?

Chatbot reply:

¿Dónde está México?

Goodbye


In [40]:
# translate to any language
model = ChatOpenAI(model_name="gpt-4o-mini")

# change in the prompt
# add a variable called language to the prompt template
prompt = ChatPromptTemplate(
    [
        ("system", "Translate the input from English to {language}"),
        ("placeholder", "{messages}")
    ]
)


# %% ##################################################################
# MessagesState objects only have a single key called mesages
# we need to extend this class to also add another key language
# which will be passed to app.invoke eventually
class CustomState(MessagesState):
    language: str


# %% ##################################################################

def call_model(state: CustomState):
    chain = prompt | model
    updated_messages = chain.invoke(state)
    return {"messages": updated_messages}


workflow = StateGraph(CustomState)
workflow.add_node("model_node", call_model)
workflow.add_edge(START, "model_node")
memory = MemorySaver()
app = workflow.compile(memory)

In [43]:
def translatebot(language: str):
    # since there is no notion of conversation for translatebot
    # we will just set the chatid to a fixed number
    config = {"configurable": {"thread_id": 999}}

    while True:
        user_input = input("User: ")
        if user_input in ('exit', 'quit'):
            print("Goodbye")
            break
        else:
            print("Chatbot reply:\n")
            for chunk, metadata in app.stream({'messages': user_input, "language": language},
                                              config=config,
                                              stream_mode="messages"):
                print(chunk.content, end="", flush=True)
            print("\n")

In [44]:
translatebot("Bengali")

Chatbot reply:

আমি ইমুকে ভালোবাসি!

Chatbot reply:

তুমি কোথায় ইমু? দয়া করে আমার কাছে এসো!!!

Goodbye


# Updating App with real time search

In [3]:

def call_model(state: MessagesState):
    updated_messages = model.invoke(state["messages"])

    # models response appended to the existing list of messages
    return {"messages": updated_messages}

In [4]:
def chatbot(chat_id: int):
    # this config dict is required because we are using a checkpointer
    config = {"configurable": {"thread_id": chat_id}}

    while True:
        user_input = input("User: ")
        if user_input in ('exit', 'quit'):
            print("Goodbye")
            break
        else:
            print("Chatbot reply:\n")
            for chunk, metadata in app.stream({'messages': user_input},
                                              config=config,
                                              stream_mode="messages"):
                print(chunk.content, end="", flush=True)
            print("\n")

Tools allow LLMs to perform actions and retrieve data from external sources. Tools is what we use to extend the capabilities of Langchain based apps. First tool we use is tavily-search. Search engine for LLMs. We need to create an account in Tavily and create API-key.

You tell the model that this tool is availabel for use, and you'll tell the model whatthe tool does. for a given input to the model, the model itself decides if it has to use the tool.

In [5]:
load_dotenv()

True

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=5)
tool_list = [search]

In [7]:
model = ChatOpenAI(model_name="gpt-4o-mini").bind_tools(tool_list)

In [8]:
from langgraph.prebuilt import ToolNode, tools_condition

call_tool = ToolNode(tool_list)

In [9]:
workflow = StateGraph(MessagesState)
workflow.add_node("model_node", call_model)
workflow.add_node("tools", call_tool)

workflow.add_edge(START, "model_node")
workflow.add_conditional_edges("model_node", tools_condition)
workflow.add_edge("tools", "model_node")

memory = MemorySaver()
app = workflow.compile(memory)

In [10]:
chatbot(4)

Chatbot reply:

[{"title": "OceanGate - Wikipedia", "url": "https://en.wikipedia.org/wiki/OceanGate", "content": "OceanGate Inc. is an American privately owned company based in Everett, Washington, that provided crewed submersibles for tourism, industry, research, and exploration. The company was founded in 2009 by Stockton Rush and Guillermo Söhnlein. [...] OceanGate was initiated by Guillermo Söhnlein and Stockton Rush in Seattle in 2009.[17][18][19][20] According to Söhnlein, the company was founded with the intention of creating a small fleet of 5-person commercial submersibles that could be leased by any organization or group of individuals. In 2023 he told Sky News, \"The whole intent was to create a small fleet of work submersibles. And in that way, as our tagline was in the early days, 'Open the oceans for all of humanity.'\"[21] [...] The company acquired a submersible vessel, Antipodes, and later built two of its own: Cyclops 1 and Titan. In 2021, OceanGate began taking payin

In [11]:
from langchain_core.messages import AIMessage, ToolMessage
import json


def chatbot(chat_id: int):
    # this config dict is required because we are using a checkpointer
    config = {"configurable": {"thread_id": chat_id}}

    while True:
        user_input = input("User: ")
        if user_input in ('exit', 'quit'):
            print("Goodbye")
            break
        else:
            print("Chatbot reply:\n")
            for chunk, metadata in app.stream({'messages': user_input},
                                              config=config,
                                              stream_mode="messages"):
                if isinstance(chunk, AIMessage):
                    print(chunk.content, end="", flush=True)
                if isinstance(chunk, ToolMessage):
                    result_list = json.loads(chunk.content)
                    print(result_list[0])
            print("\n")

# RAG
It is a method of providing LLM with data that it was not trained on, in order to improve its results. This could be the case when we want to give the model data that occurred after its training cutoff date. Training cutoff of gopt-4 mini is October 2023

In [12]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.python.org/3/whatsnew/3.13.html")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [13]:
# docs is a list of one document corresponding to one url
docs = loader.load()

In [14]:
docs[0].metadata

{'source': 'https://docs.python.org/3/whatsnew/3.13.html',
 'title': 'What’s New In Python 3.13 — Python 3.13.3 documentation',
 'description': 'Editors, Adam Turner and Thomas Wouters,. This article explains the new features in Python 3.13, compared to 3.12. Python 3.13 was released on October 7, 2024. For full details, see the changelog. ...',
 'language': 'en'}

In [17]:
print(docs[0].page_content[:2500])


















What’s New In Python 3.13 — Python 3.13.3 documentation




















































    Theme
    
Auto
Light
Dark



Table of Contents

What’s New In Python 3.13
Summary – Release Highlights
New Features
A better interactive interpreter
Improved error messages
Free-threaded CPython
An experimental just-in-time (JIT) compiler
Defined mutation semantics for locals()
Support for mobile platforms


Other Language Changes
New Modules
Improved Modules
argparse
array
ast
asyncio
base64
compileall
concurrent.futures
configparser
copy
ctypes
dbm
dis
doctest
email
enum
fractions
glob
importlib
io
ipaddress
itertools
marshal
math
mimetypes
mmap
multiprocessing
os
os.path
pathlib
pdb
queue
random
re
shutil
site
sqlite3
ssl
statistics
subprocess
sys
tempfile
time
tkinter
traceback
types
typing
unicodedata
venv
warnings
xml
zipimport


Optimizations
Removed Modules And APIs
PEP 594: Remove “dead batteries” from the standard library
2to3
builtins
configparser

In [18]:
# RAG splitting
from langchain_text_splitters import RecursiveCharacterTextSplitter

# chunk size are in characters
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [19]:
chunks = text_splitter.split_documents(docs)
len(chunks)

153

In [20]:
print(chunks[0].page_content)

What’s New In Python 3.13 — Python 3.13.3 documentation




















































    Theme
    
Auto
Light
Dark



Table of Contents

What’s New In Python 3.13
Summary – Release Highlights
New Features
A better interactive interpreter
Improved error messages
Free-threaded CPython
An experimental just-in-time (JIT) compiler
Defined mutation semantics for locals()
Support for mobile platforms


Other Language Changes
New Modules
Improved Modules
argparse
array
ast
asyncio
base64
compileall
concurrent.futures
configparser
copy
ctypes
dbm
dis
doctest
email
enum
fractions
glob
importlib
io
ipaddress
itertools
marshal
math
mimetypes
mmap
multiprocessing
os
os.path
pathlib
pdb
queue
random
re
shutil
site
sqlite3
ssl
statistics
subprocess
sys
tempfile
time
tkinter
traceback
types
typing
unicodedata
venv
warnings
xml
zipimport


In [25]:
# embed the chunks into vectors
# embeddings are fixed length array of numbers
# this length is defined by the embedding model
# the embedding model puts documents/chunks with similar meanings in the same location
# we can calculate the semantic similarity between pairs of docs using math

from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

In [26]:
# for now we will use an in memoery vector store which will not persist
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore.from_documents(chunks, embedding_model)

In [29]:
# lets create the retriever
retriever = vector_store.as_retriever()
results = retriever.invoke("new interactive interpreter")

In [30]:
# by default returns 4 chunks but can be configured
len(results)

4

In [31]:
for result in results:
    print(result.page_content)
    print("-----")

Multiline editing with history preservation.
Direct support for REPL-specific commands like help, exit,
and quit, without the need to call them as functions.
Prompts and tracebacks with color enabled by default.
Interactive help browsing using F1 with a separate command
history.
History browsing using F2 that skips output as well as the
>>> and … prompts.
“Paste mode” with F3 that makes pasting larger blocks of code
easier (press F3 again to return to the regular prompt).

To disable the new interactive shell,
set the PYTHON_BASIC_REPL environment variable.
For more on interactive mode, see Interactive Mode.
(Contributed by Pablo Galindo Salgado, Łukasz Langa, and
Lysandros Nikolaou in gh-111201 based on code from the PyPy project.
Windows support contributed by Dino Viehland and Anthony Shaw.)


Improved error messages¶
-----
Release schedule changes:
PEP 602 (“Annual Release Cycle for Python”) has been updated
to extend the full support (‘bugfix’) period for new releases to two years

In [33]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever, "python_3_13_update_retriever",
                                       "A retriever that returns relevant documents from the What's New page of the Python 3.13 documentation. "
                                       "Useful when you need to answer questions about the features, enhancements, removals, deprecations, and "
                                       "others changes introduced in this version of Python."
                                       )

# incorporate the retriever

In [63]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AIMessage, ToolMessage
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.tools.retriever import create_retriever_tool
import json

In [64]:
def call_model(state: MessagesState):
    updated_messages = model.invoke(state["messages"])

    # models response appended to the existing list of messages
    return {"messages": updated_messages}

In [65]:
def chatbot(chat_id: int):
    # this config dict is required because we are using a checkpointer
    config = {"configurable": {"thread_id": chat_id}}

    while True:
        user_input = input("User: ")
        if user_input in ('exit', 'quit'):
            print("Goodbye")
            break
        else:
            print("Chatbot reply:\n")
            for chunk, metadata in app.stream({'messages': user_input},
                                              config=config,
                                              stream_mode="messages"):
                print(chunk.content, end="", flush=True)
            print("\n")

In [66]:
def chatbot(chat_id: int):
    # this config dict is required because we are using a checkpointer
    config = {"configurable": {"thread_id": chat_id}}

    while True:
        user_input = input("User: ")
        if user_input in ('exit', 'quit'):
            print("Goodbye")
            break
        else:
            print("Chatbot reply:\n")
            for chunk, metadata in app.stream({'messages': user_input},
                                              config=config,
                                              stream_mode="messages"):
                if isinstance(chunk, AIMessage):
                    print(chunk.content, end="", flush=True)
                if isinstance(chunk, ToolMessage):
                    result_list = json.loads(chunk.content)
                    print(result_list[0])
            print("\n")

In [67]:
def chatbot(chat_id: int):
    # this config dict is required because we are using a checkpointer
    config = {"configurable": {"thread_id": chat_id}}

    while True:
        user_input = input("User: ")
        if user_input in ('exit', 'quit'):
            print("Goodbye")
            break
        else:
            print("Chatbot reply:\n")
            for chunk, metadata in app.stream({'messages': user_input},
                                              config=config,
                                              stream_mode="messages"):
                if isinstance(chunk, AIMessage):
                    print(chunk.content, end="", flush=True)
                if isinstance(chunk, ToolMessage):
                    if chunk.name == 'tavily_search_results_json':
                        print("Using Tavily search tool")
                        result_list = json.loads(chunk.content)
                        urls = [result["url"] for result in result_list]
                        print(urls, end="\n\n")
                    if chunk.name == 'python_3_13_update_retriever':
                        print("Checking Python documentation using RAG...", end="\n\n")
            print("\n")

In [68]:
load_dotenv()

loader = WebBaseLoader("https://www.linkedin.com/in/tasneemfatema/")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(docs)
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = InMemoryVectorStore.from_documents(chunks, embedding_model)
retriever = vector_store.as_retriever()
retriever_tool = create_retriever_tool(retriever, "python_3_13_update_retriever",
                                       "A retriever that returns relevant documents from the What's New page of the Python 3.13 documentation. "
                                       "Useful when you need to answer questions about the features, enhancements, removals, deprecations, and "
                                       "others changes introduced in this version of Python."
                                       )

search = TavilySearchResults(max_results=5)
tool_list = [retriever_tool, search]
model = ChatOpenAI(model_name="gpt-4o-mini").bind_tools(tool_list)
call_tool = ToolNode(tool_list)

workflow = StateGraph(MessagesState)
workflow.add_node("model_node", call_model)
workflow.add_node("tools", call_tool)
workflow.add_edge(START, "model_node")
workflow.add_conditional_edges("model_node", tools_condition)
workflow.add_edge("tools", "model_node")

memory = MemorySaver()
app = workflow.compile(memory)

In [74]:
chatbot(99_999)

Chatbot reply:

Using Tavily search tool
['https://bd.linkedin.com/in/nafe-muhtasim-hye', 'https://orcid.org/0009-0005-7978-4622', 'https://www.researchgate.net/profile/Nafe-Hye', 'https://ieeexplore.ieee.org/author/37089820409', 'https://aust.academia.edu/NafeMuhtasimHye']

Nafe Muhtasim Hye is a professional with a background in electrical and electronics engineering. He obtained his BSc degree from Ahsanullah University of Science and Technology (AUST) in 2021. His work focuses on addressing critical challenges in the biomedical field through the integration of artificial intelligence and machine learning.

He has been involved in significant projects, including being part of Bangladesh's pioneering MARS Rover team during his studies. Currently, he serves as the Chief Operating Officer at Consel Health.

For more detailed information, you can check his [LinkedIn profile](https://bd.linkedin.com/in/nafe-muhtasim-hye) or [ORCID page](https://orcid.org/0009-0005-7978-4622).

Goodbye
